In [ ]:
%pip install jupyter_http_over_ws
!jupyter serverextension enable --py jupyter_http_over_ws

In [ ]:
!jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
import numpy as np
import sys
import pandas as pd

sys.setrecursionlimit(10**8)

class GameState():
	def __init__(self):
		self.board = np.array([
			['bR','bN','bB','bQ','bK','bB','bN','bR'],
			['bP','bP','bP','bP','bP','bP','bP','bP'],
			['--','--','--','--','--','--','--','--'],
			['--','--','--','--','--','--','--','--'],
			['--','--','--','--','--','--','--','--'],
			['--','--','--','--','--','--','--','--'],
			['wP','wP','wP','wP','wP','wP','wP','wP'],
			['wR','wN','wB','wQ','wK','wB','wN','wR']
		])
		self.draw = [
			['wK', 'bK'], 

			['wK', 'bK', 'bN', 'bN'], 
			['wK', 'bK', 'wN', 'wN'], 
			['wK', 'bK', 'bN', 'wN'], 
			['wK', 'bK', 'wN', 'bN'], 

			['wK', 'bK', 'bN', 'wB'], 
			['wK', 'bK', 'wN', 'bB'], 
			['wK', 'bK', 'wN', 'wB'], 
			['wK', 'bK', 'bN', 'bB'], 

			['wK', 'bK', 'bN'], 
			['wK', 'bK', 'wN'], 

			['wK', 'bK', 'wB'],
			['wK', 'bK', 'bB']
		]
		self.whiteToMove = True
		self.moveLog = []
		self.undoneMoves = []
		self.whiteKingLocation = (7, 4)
		self.blackKingLocation = (0, 4)
		self.checkmate = False
		self.stalemate = False
		self.notEnoughMaterialsToWin = False
		self.winner = ""
		self.inCheck = False
		self.checks = []
		self.pins = []
		self.enpassantPossible = ()
		self.enpassantPossibleLog = [self.enpassantPossible]
		self.currentCastelingRight = CastleRights(True, True, True, True)
		self.castleRightLog = [CastleRights(self.currentCastelingRight.wks, self.currentCastelingRight.bks, 
									self.currentCastelingRight.wqs, self.currentCastelingRight.bqs)]

	def makeMove(self, move, promotionTo = 'Q'):
		self.board[move.startRow][move.startCol] = '--'
		self.board[move.endRow][move.endCol] = move.pieceMoved
		self.moveLog.append(move)
		self.whiteToMove = not self.whiteToMove

		if move.pieceMoved == 'wK':
			self.whiteKingLocation = (move.endRow, move.endCol)
		if move.pieceMoved == 'bK':
			self.blackKingLocation = (move.endRow, move.endCol)

		if move.pawnPromotion == True:
			self.board[move.endRow][move.endCol] = move.pieceMoved[0] + promotionTo

		if move.enpassantMove == True:
			self.board[move.startRow][move.endCol] = '--'
			
		if move.pieceMoved[1] == 'P' and abs(move.startRow - move.endRow) == 2:
			self.enpassantPossible = ((move.startRow + move.endRow)//2, move.endCol)
		else:
			self.enpassantPossible = ()

		if move.castleMove == True:
			if move.endCol - move.startCol == 2:
				self.board[move.endRow][move.endCol-1] = self.board[move.endRow][move.endCol+1]
				self.board[move.endRow][move.endCol+1] = '--'
			else:
				self.board[move.endRow][move.endCol+1] = self.board[move.endRow][move.endCol-2]
				self.board[move.endRow][move.endCol-2] = '--'

		self.updateCastleRights(move)
		self.castleRightLog.append(CastleRights(self.currentCastelingRight.wks, self.currentCastelingRight.bks, 
									self.currentCastelingRight.wqs, self.currentCastelingRight.bqs))

		self.enpassantPossibleLog.append(self.enpassantPossible)

	def undoMove(self):
		if len(self.moveLog) != 0:
			move = self.moveLog.pop()
			self.undoneMoves.append(move)
			self.board[move.startRow][move.startCol] = move.pieceMoved
			self.board[move.endRow][move.endCol] = move.pieceCaptured
			self.whiteToMove = not self.whiteToMove

			if move.pieceMoved == 'wK':
				self.whiteKingLocation = (move.startRow, move.startCol)
			if move.pieceMoved == 'bK':
				self.blackKingLocation = (move.startRow, move.startCol)

			if move.enpassantMove == True:
				self.board[move.endRow][move.endCol] = '--'
				self.board[move.startRow][move.endCol] = move.pieceCaptured
				
			self.enpassantPossibleLog.pop()
			self.enpassantPossible = self.enpassantPossibleLog[-1]

			self.castleRightLog.pop()
			self.currentCastelingRight = self.castleRightLog[-1]

			if move.castleMove == True:
				if move.endCol - move.startCol == 2:
					self.board[move.endRow][move.endCol+1] = self.board[move.endRow][move.endCol-1]
					self.board[move.endRow][move.endCol-1] = '--'
				else:
					self.board[move.endRow][move.endCol-2] = self.board[move.endRow][move.endCol+1]
					self.board[move.endRow][move.endCol+1] = '--'

			self.checkmate = False
			self.stalemate = False

	def updateCastleRights(self, move):
		if move.pieceMoved == 'wK':
			self.currentCastelingRight.wks = False
			self.currentCastelingRight.wqs = False
		elif move.pieceMoved == 'bK':
			self.currentCastelingRight.bks = False
			self.currentCastelingRight.bqs = False
		elif move.pieceMoved == 'wR':
			if move.startCol == 0:
				self.currentCastelingRight.wqs = False
			elif move.startCol == 7:
				self.currentCastelingRight.wks = False
		elif move.pieceMoved == 'bR':
			if move.startCol == 0:
				self.currentCastelingRight.bqs = False
			elif move.startCol == 7:
				self.currentCastelingRight.bks = False

		if move.pieceCaptured == 'bR':
			if move.endRow == 0:
				if move.endCol == 0:
					self.currentCastelingRight.bqs = False
				elif move.endCol == 7:
					self.currentCastelingRight.bks = False
		elif move.pieceCaptured == 'wR':
			if move.endRow == 7:
				if move.endCol == 0:
					self.currentCastelingRight.wqs = False
				elif move.endCol == 7:
					self.currentCastelingRight.wks = False

	def redoMove(self, promotionTo=None):
		if len(self.undoneMoves) != 0:
			move = self.undoneMoves.pop()
			self.makeMove(move, promotionTo)

	def getValidMoves(self):
		tempEnpassantPossible = self.enpassantPossible

		moves = []
		self.inCheck, self.pins, self.checks = self.checkForPinsAndChecks()
		if self.whiteToMove:
			kingRow = self.whiteKingLocation[0]
			kingCol = self.whiteKingLocation[1]
		else:
			kingRow = self.blackKingLocation[0]
			kingCol = self.blackKingLocation[1]
		if self.inCheck == True:
			if len(self.checks) == 1:
				moves = self.getPossibleMoves()
				check = self.checks[0]
				checkRow = check[0]
				checkCol = check[1]
				pieceChecking = self.board[checkRow][checkCol]
				validSquares = []
				if pieceChecking[1] == 'N':
					validSquares = [(checkRow, checkCol)]
				else:
					for i in range(1, 8):
						validSquare = (kingRow + check[2] * i, kingCol + check[3] * i)
						validSquares.append(validSquare)
						if validSquare[0] == checkRow and validSquare[1] == checkCol:
							break
				for i in range(len(moves)-1, -1, -1):
					if moves[i].pieceMoved[1] != 'K':
						if not (moves[i].endRow, moves[i].endCol) in validSquares:
							moves.remove(moves[i])
			else:
				self.getKingMoves(kingRow, kingCol, moves)
		else:
			moves = self.getPossibleMoves()
			
		if moves == [] and self.inCheck == True:
			self.checkmate = True
		elif moves == [] and self.inCheck == False:
			self.stalemate = True

		if self.checkmate == True:
			self.winner = "bK" if self.whiteToMove else "wK"

		self.enpassantPossible = tempEnpassantPossible

		piecesLeft = []
		for row in self.board:
			for square in row:
				if square != '--':
					piecesLeft.append(square)
		for pl in self.draw:
			if pl.sort() == piecesLeft.sort() and len(pl) == len(piecesLeft):
				self.notEnoughMaterialsToWin = True
		
		return moves

	def squareUnderAttack(self, r, c):
		self.whiteToMove = not self.whiteToMove
		oppMoves = self.getPossibleMoves()
		self.whiteToMove = not self.whiteToMove
		for move in oppMoves:
			if move.endRow == r and move.endCol == c:
				return True
		return False

	def checkForPinsAndChecks(self):
		pins = []
		checks = []
		inCheck = False
		if self.whiteToMove == True:
			enemyColor = 'b'
			allyColor = 'w'
			startRow = self.whiteKingLocation[0]
			startCol = self.whiteKingLocation[1]
		else:
			enemyColor = 'w'
			allyColor = 'b'
			startRow = self.blackKingLocation[0]
			startCol = self.blackKingLocation[1]

		directions = ((-1,0), (0,-1), (1,0), (0,1), (-1,-1), (-1,1), (1,-1), (1,1))
		for j in range(len(directions)):
			d = directions[j]
			possiblePin = ()
			for i in range(1, 8):
				endRow = startRow + d[0] * i
				endCol = startCol + d[1] * i
				if 0 <= endRow < 8 and 0 <= endCol < 8:
					endPiece = self.board[endRow][endCol]
					if endPiece[0] == allyColor and endPiece[1] != 'K':
						if possiblePin == ():
							possiblePin = (endRow, endCol, d[0], d[1])
						else:
							break
					elif endPiece[0] == enemyColor:
						type = endPiece[1]
						if (0 <= j <= 3 and type == 'R') or \
								(4 <= j <= 7 and type == 'B') or \
								(i == 1 and type == 'P' and \
								(\
									(enemyColor == 'w' and 6 <= j <= 7) or \
									(enemyColor == 'b' and 4 <= j <= 5)\
								)) or \
								(type == 'Q') or \
								(i == 1 and type == 'K'):
							if possiblePin == ():
								inCheck = True
								checks.append((endRow, endCol, d[0], d[1]))
								break
							else:
								pins.append(possiblePin)
								break
						else:
							break
		knightmoves = ((-2,-1), (-2,1), (-1,-2), (-1,2), (1,-2), (1,2), (2,-1), (2,1))
		for m in knightmoves:
			endRow = startRow + m[0]
			endCol = startCol + m[1]
			if 0 <= endRow < 8 and 0 <= endCol < 8:
				endPiece = self.board[endRow][endCol]
				if endPiece[0] == enemyColor and endPiece[1] == 'N':
					inCheck = True
					checks.append((endRow, endCol, m[0], m[1]))
		return inCheck, pins, checks

	def getPossibleMoves(self):
		moves = []
		for r in range(len(self.board)):
			for c in range(len(self.board[r])):
				turn = self.board[r][c][0]
				if (turn == 'w' and self.whiteToMove == True) or (turn == 'b' and self.whiteToMove == False):
					piece = self.board[r][c][1]
					if piece == 'P':
						self.getPawnMoves(r, c, moves)
					elif piece == 'R':
						self.getRookMoves(r, c, moves)
					elif piece == 'N':
						self.getKnightMoves(r, c, moves)
					elif piece == 'B':
						self.getBishopMoves(r, c, moves)
					elif piece == 'Q':
						self.getQueenMoves(r, c, moves)
					elif piece == 'K':
						self.getKingMoves(r, c, moves)
						self.getCastleMoves(self.whiteKingLocation[0] if self.whiteToMove == True else self.blackKingLocation[0], self.whiteKingLocation[1] if self.whiteToMove == True else self.blackKingLocation[1], moves, ('w' if self.whiteToMove else 'b'))
		return moves

	def getPawnMoves(self, r, c, moves):
		piecePinned = False
		pinDirection = ()
		for i in range(len(self.pins)-1, -1, -1):
			if self.pins[i][0] == r and self.pins[i][1] == c:
				piecePinned = True
				pinDirection = (self.pins[i][2], self.pins[i][3])
				self.pins.remove(self.pins[i])
				break
			
		kingRow, kingCol = (self.whiteKingLocation if self.whiteToMove == True else self.blackKingLocation)

		if self.whiteToMove == True:
			if self.board[r-1][c] == '--':
				if piecePinned == False or pinDirection == (-1, 0):
					moves.append(Move((r, c), (r-1, c), self.board))
					if r == 6 and self.board[r-2][c] == '--':
						moves.append(Move((r, c), (r-2, c), self.board))
			if c-1 >= 0:
				if piecePinned == False or pinDirection == (-1, -1):
					if self.board[r-1][c-1][0] == 'b':
						moves.append(Move((r, c), (r-1, c-1), self.board))
					elif (r-1, c-1) == self.enpassantPossible:
						attackPiece = blockPiece = False
						if kingRow == r:
							if kingCol < c:
								insideRange = [*range(kingCol+1, c-1)]
								outsideRange = [*range(c+1, 8)]
							else:
								insideRange = [*range(kingCol-1, c, -1)]
								outsideRange = [*range(c-2, -1, -1)]
							for i in insideRange:
								if self.board[r][i] != '--':
									blockPiece = True
							for i in outsideRange:
								square = self.board[r][i]
								if (square[0] == 'b' if self.whiteToMove == True else 'w') and (square[1] == 'R' or square[1] == 'Q'):
									attackPiece = False
								if square[0] != '--':
									blockPiece = True
						if blockPiece == True or attackPiece == False:
							moves.append(Move((r, c), (r-1, c-1), self.board, enpassant=True))
			if c+1 <= 7:
				if piecePinned == False or pinDirection == (-1, 1):
					if self.board[r-1][c+1][0] == 'b':
						moves.append(Move((r, c), (r-1, c+1), self.board))
					elif (r-1, c+1) == self.enpassantPossible:
						attackPiece = blockPiece = False
						if kingRow == r:
							if kingCol < c:
								insideRange = [*range(kingCol+1, c)]
								outsideRange = [*range(c+2, 8)]
							else:
								insideRange = [*range(kingCol-1, c, -1)]
								outsideRange = [*range(c-2, -1, -1)]
							for i in insideRange:
								if self.board[r][i] != '--':
									blockPiece = True
							for i in outsideRange:
								square = self.board[r][i]
								if (square[0] == 'b' if self.whiteToMove == True else 'w') and (square[1] == 'R' or square[1] == 'Q'):
									attackPiece = False
								if square[0] != '--':
									blockPiece = True
						if blockPiece == True or attackPiece == False:
							moves.append(Move((r, c), (r-1, c+1), self.board, enpassant=True))
		else:
			if self.board[r+1][c] == '--':
				if piecePinned == False or pinDirection == (1, 0):
					moves.append(Move((r, c), (r+1, c), self.board))
					if r == 1 and self.board[r+2][c] == '--':
						moves.append(Move((r, c), (r+2, c), self.board))
			if c-1 >= 0:
				if piecePinned == False or pinDirection == (1, -1):
					if self.board[r+1][c-1][0] == 'w':
						moves.append(Move((r, c), (r+1, c-1), self.board))
					elif (r+1, c-1) == self.enpassantPossible:
						attackPiece = blockPiece = False
						if kingRow == r:
							if kingCol < c:
								insideRange = [*range(kingCol+1, c-1)]
								outsideRange = [*range(c+1, 8)]
							else:
								insideRange = [*range(kingCol-1, c, -1)]
								outsideRange = [*range(c-2, -1, -1)]
							for i in insideRange:
								if self.board[r][i] != '--':
									blockPiece = True
							for i in outsideRange:
								square = self.board[r][i]
								if (square[0] == 'b' if self.whiteToMove == True else 'w') and (square[1] == 'R' or square[1] == 'Q'):
									attackPiece = False
								if square[0] != '--':
									blockPiece = True
						if blockPiece == True or attackPiece == False:
							moves.append(Move((r, c), (r+1, c-1), self.board, enpassant=True))
			if c+1 <= 7:
				if piecePinned == False or pinDirection == (1, 1):
					if self.board[r+1][c+1][0] == 'w':
						moves.append(Move((r, c), (r+1, c+1), self.board))
					elif (r+1, c+1) == self.enpassantPossible:
						attackPiece = blockPiece = False
						if kingRow == r:
							if kingCol < c:
								insideRange = [*range(kingCol+1, c)]
								outsideRange = [*range(c+2, 8)]
							else:
								insideRange = [*range(kingCol-1, c, -1)]
								outsideRange = [*range(c-2, -1, -1)]
							for i in insideRange:
								if self.board[r][i] != '--':
									blockPiece = True
							for i in outsideRange:
								square = self.board[r][i]
								if (square[0] == 'b' if self.whiteToMove == True else 'w') and (square[1] == 'R' or square[1] == 'Q'):
									attackPiece = False
								if square[0] != '--':
									blockPiece = True
						if blockPiece == True or attackPiece == False:
							moves.append(Move((r, c), (r+1, c+1), self.board, enpassant=True))
	
	def getRookMoves(self, r, c, moves):
		piecePinned = False
		pinDirection = ()
		for i in range(len(self.pins)-1, -1, -1):
			if self.pins[i][0] == r and self.pins[i][1] == c:
				piecePinned = True
				pinDirection = (self.pins[i][2], self.pins[i][3])
				if self.board[r][c][1] != 'Q':
					self.pins.remove(self.pins[i])
				break
			
		directions = ((-1,0),(0,-1),(1,0),(0,1))
		enemycolor = 'b' if self.whiteToMove else 'w'
		for d in directions:
			for i in range(1,8):
				endRow = r + d[0] * i
				endCol = c + d[1] * i
				if 0 <= endRow < 8 and 0 <= endCol < 8:
					if piecePinned == False or pinDirection == d or pinDirection == (-d[0], -d[1]):
						endPiece = self.board[endRow][endCol]
						if endPiece == '--':
							moves.append(Move((r, c), (endRow, endCol), self.board))
						elif endPiece[0] == enemycolor:
							moves.append(Move((r, c), (endRow, endCol), self.board))
							break
						else:
							break
				else:
					break
			
	def getKnightMoves(self, r, c, moves):
		piecePinned = False
		for i in range(len(self.pins)-1, -1, -1):
			if self.pins[i][0] == r and self.pins[i][1] == c:
				piecePinned = True
				self.pins.remove(self.pins[i])
				break
			
		knightmoves = ((-2,-1), (-2,1), (-1,-2), (-1,2), (1,-2), (1,2), (2,-1), (2,1))
		allycolor = 'w' if self.whiteToMove else 'b'
		for m in knightmoves:
			endRow = r + m[0]
			endCol = c + m[1]
			if 0 <= endRow < 8 and 0 <= endCol < 8:
				if piecePinned == False:
					endPiece = self.board[endRow][endCol]
					if endPiece[0] != allycolor:
						moves.append(Move((r, c), (endRow, endCol), self.board))

	def getBishopMoves(self, r, c, moves):
		piecePinned = False
		pinDirection = ()
		for i in range(len(self.pins)-1, -1, -1):
			if self.pins[i][0] == r and self.pins[i][1] == c:
				piecePinned = True
				pinDirection = (self.pins[i][2], self.pins[i][3])
				self.pins.remove(self.pins[i])
				break
			
		directions = ((-1,-1),(1,1),(1,-1),(-1,1))
		enemycolor = 'b' if self.whiteToMove else 'w'
		for d in directions:
			for i in range(1,8):
				endRow = r + d[0] * i
				endCol = c + d[1] * i
				if 0 <= endRow < 8 and 0 <= endCol < 8:
					if piecePinned == False or pinDirection == d or pinDirection == (-d[0], -d[1]):
						endPiece = self.board[endRow][endCol]
						if endPiece == '--':
							moves.append(Move((r, c), (endRow, endCol), self.board))
						elif endPiece[0] == enemycolor:
							moves.append(Move((r, c), (endRow, endCol), self.board))
							break
						else:
							break
				else:
					break

	def getQueenMoves(self, r, c, moves):
		piecePinned = False
		pinDirection = ()
		for i in range(len(self.pins)-1, -1, -1):
			if self.pins[i][0] == r and self.pins[i][1] == c:
				piecePinned = True
				pinDirection = (self.pins[i][2], self.pins[i][3])
				self.pins.remove(self.pins[i])
				break
			
		self.getRookMoves(r, c, moves)
		self.getBishopMoves(r, c, moves)
	
	def getKingMoves(self, r, c, moves):
		rowmoves = (-1, -1, -1, 0, 0, 1, 1, 1)
		colmoves = (-1, 0, 1, -1, 1, -1, 0, 1)
		allyColor = 'w' if self.whiteToMove else 'b'
		for i in range(8):
			endRow = r + rowmoves[i]
			endCol = c + colmoves[i]
			if 0 <= endRow < 8 and 0 <= endCol < 8:
				endPiece = self.board[endRow][endCol]
				if endPiece[0] != allyColor:
					if allyColor == 'w':
						self.whiteKingLocation = (endRow, endCol)
					else:
						self.blackKingLocation = (endRow, endCol)
					inCheck, pins, checks = self.checkForPinsAndChecks()
					if inCheck == False:
						moves.append(Move((r, c), (endRow, endCol), self.board))
					if allyColor == 'w':
						self.whiteKingLocation = (r, c)
					else:
						self.blackKingLocation = (r, c)

	def getCastleMoves(self, r, c, moves, allyColor):
		if self.inCheck == True:
			return []
		if (self.whiteToMove == True and self.currentCastelingRight.wks == True) or (self.whiteToMove == False and self.currentCastelingRight.bks == True):
			self.getKingsideCastleMoves(r, c, moves, allyColor)
		if (self.whiteToMove == True and self.currentCastelingRight.wqs == True) or (self.whiteToMove == False and self.currentCastelingRight.bqs == True):
			self.getQueensideCastleMoves(r, c, moves, allyColor)
		return moves
		
	def getKingsideCastleMoves(self, r, c, moves, allyColor):
		if self.board[r][c+1] == '--' and self.board[r][c+2] == '--':
			if not self.squareUnderAttack(r, c+1) and not self.squareUnderAttack(r, c+2):
				moves.append(Move((r, c), (r, c+2), self.board, castle=True))

	def getQueensideCastleMoves(self, r, c, moves, allyColor):
		if self.board[r][c-1] == '--' and self.board[r][c-2] == '--' and self.board[r][c-3] == '--':
			if not self.squareUnderAttack(r, c-1) and not self.squareUnderAttack(r, c-2) and not self.squareUnderAttack(r, c-3):
				moves.append(Move((r, c), (r, c-2), self.board, castle=True))
		
	def getProtections(self, row, col, validMoves, AI):
		protections = 0
		for m in validMoves:
			if m.endCol == col and m.endRow == row:
				protections += float(AI.iat[6,1])
		return protections
	
	def getValidMoveForSquare(self, row, col, validMoves, AI):
		validMovesForSquare = 0
		for m in validMoves:
			if m.startCol == col and m.startRow == row:
				validMovesForSquare += float(AI.iat[7,1])
		return validMovesForSquare

	def getAttacks(self, row, col, validMoves, AI):
		attacks = 0
		for m in validMoves:
			if m.startCol == col and m.startRow == row and m.pieceCaptured == 'w' if self.board[row][col][0] == 'b' else 'b':
				attacks += float(AI.iat[8,1])
		return attacks
	
	def castle(self, validMoves, AI):
		for m in validMoves:
			if m.castleMove == True:
				return float(AI.iat[9,1])
		return 0

	def getPositionAdvantages(self, turn, AI):
		position = 0
		if turn == 'w':
			if self.board[7][3] != 'wQ' and len(self.moveLog) < 10*2:
				position -= float(AI.iat[10,1])
			if self.board[7][4] != 'wK' and len(self.moveLog) < 20*2:
				position -= float(AI.iat[11,1])
			''' if self.board[7][4] != 'wK' and len(self.moveLog) < 4:
				position += 20 ''' #Bongcloud
			if self.board[4][3] == 'wP' or self.board[4][3] == 'wN' or \
				self.board[4][4] == 'wP' or self.board[4][4] == 'wN' or \
				self.board[3][3] == 'wP' or self.board[3][3] == 'wN' or \
				self.board[3][4] == 'wP' or self.board[3][4] == 'wN':
				position += float(AI.iat[12,1])
			if (self.board[7][1] == 'wN' or self.board[7][2] == 'wB' or \
				self.board[7][5] == 'wB' or self.board[7][6] == 'wN') and len(self.moveLog) > 6*2:
				position -= float(AI.iat[13,1])
			if self.board[4][2] == 'wB' or self.board[3][1] == 'wB' or \
				self.board[4][5] == 'wB' or self.board[3][6] == 'wB':
				position += float(AI.iat[14,1])
			if self.board[5][2] == 'wN' or self.board[5][5] == 'wN':
				position += float(AI.iat[15,1])
			if self.board[5][0] == 'wN' or self.board[5][7] == 'wN':
				position -= float(AI.iat[16,1])
		else:
			if self.board[0][3] != 'bQ' and len(self.moveLog) < 10*2:
				position -= float(AI.iat[10,1])
			if self.board[0][4] != 'bK' and len(self.moveLog) < 20*2:
				position -= float(AI.iat[11,1])
			''' if self.board[0][4] != 'bK' and len(self.moveLog) < 4:
				position += 20 ''' #Bongcloud
			if self.board[4][3] == 'bP' or self.board[4][3] == 'bN' or \
				self.board[4][4] == 'bP' or self.board[4][4] == 'bN' or \
				self.board[3][3] == 'bP' or self.board[3][3] == 'bN' or \
				self.board[3][4] == 'bP' or self.board[3][4] == 'bN':
				position += float(AI.iat[12,1])
			if (self.board[0][1] == 'bN' or self.board[0][2] == 'bB' or \
				self.board[0][5] == 'bB' or self.board[0][6] == 'bN') and len(self.moveLog) > 6*2:
				position -= float(AI.iat[13,1])
			if self.board[3][2] == 'bB' or self.board[4][1] == 'bB' or \
				self.board[3][5] == 'bB' or self.board[4][6] == 'bB':
				position += float(AI.iat[14,1])
			if self.board[2][2] == 'bN' or self.board[2][5] == 'bN':
				position += float(AI.iat[15,1])
			if self.board[2][0] == 'bN' or self.board[2][7] == 'bN':
				position -= float(AI.iat[16,1])
		return position

class Move():
	ranksToRows = {'1':7,'2':6,'3':5,'4':4,
				'5':3,'6':2,'7':1,'8':0}
	rowsToRanks = {v:k for k,v in ranksToRows.items()}
	filesToCols = {'a':0,'b':1,'c':2,'d':3,
				'e':4,'f':5,'g':6,'h':7}
	colsToFiles = {v: k for k,v in filesToCols.items()}

	def __init__(self, start, end, board, enpassant=False, castle=False):
		self.startRow = start[0]
		self.startCol = start[1]
		self.endRow = end[0]
		self.endCol = end[1]
		self.pieceMoved = board[self.startRow][self.startCol]
		self.pieceCaptured = board[self.endRow][self.endCol]
		self.pawnPromotion = (self.pieceMoved == 'wP' and self.endRow == 0) or (self.pieceMoved == 'bP' and self.endRow == 7)
		self.enpassantMove = enpassant
		if self.enpassantMove == True:
			self.pieceCaptured = 'wP' if self.pieceMoved == 'bP' else 'bP'
		self.castleMove = castle
		self.moveID = self.startRow * 1000 + self.startCol * 100 + self.endRow * 10 + self.endCol

	def __eq__(self, other):
		if isinstance(other, Move):
			return self.moveID == other.moveID

	def getChessNotation(self):
		if not (abs(self.startCol - self.endCol) == 2 and self.pieceMoved[1] == 'K'):
			if self.pieceCaptured == '--':
				if self.pieceMoved[-1] != 'P':
					return self.pieceMoved[-1] + self.getRankFile(self.endRow, self.endCol)
				else:
					return self.getRankFile(self.endRow, self.endCol)
			else:
				if self.pieceMoved[-1] != 'P':
					return self.pieceMoved[-1] + "x" + self.getRankFile(self.endRow, self.endCol)
				else:
					return self.colsToFiles[self.startCol] + 'x' + self.getRankFile(self.endRow, self.endCol)
		else:
			if self.endCol - self.startCol == 2:
				return "O-O"
			else:
				return "O-O-O"
				

	def getRankFile(self, row, col):
		return self.colsToFiles[col] + self.rowsToRanks[row]

class CastleRights():
	def __init__(self, wks, bks, wqs, bqs):
		self.wks = wks
		self.bks = bks
		self.wqs = wqs
		self.bqs = bqs

In [ ]:
import random
import sys
import ChessEngine
import pandas as pd
from ChessEngine import Move

sys.setrecursionlimit(10**8)

pieceScore = {'P': 1, 'R': 5, 'N': 3, 'B': 3, 'Q': 9, 'K': 0}
CHECKMATE = 1000
STALEMATE = 0
DEPTH = 3

#White openings
Queens_gambit_white = ['d2d4', 'c2c4', 'b1c3']
Four_knights_white = ['e2e4', 'g1f3', 'b1c3']

#Black openings
Queens_gambit_black = ['e7e6', 'd7d5', 'g8f6']
Sicilian_black = ['c7c5', 'e7e6', 'b8c6', 'g8f6']
Four_knights_black = ['e7e5', 'b8c6', 'g8f6']

whiteOpeningsName = [
	'Queens_gambit_white',
	'Four_knights_white'
]
blackOpeningsName = [
	'Sicilian_black',
	'Four_knights_black',
	'Queens_gambit_black'
]
whiteOpenings = [
	Queens_gambit_white,
	Four_knights_white
]
blackOpenings = [
	Sicilian_black,
	Four_knights_black,
	Queens_gambit_black
]
chosenOpenings = []

ranksToRows = {'1':7,'2':6,'3':5,'4':4,
			'5':3,'6':2,'7':1,'8':0}
rowsToRanks = {v:k for k,v in ranksToRows.items()}
filesToCols = {'a':0,'b':1,'c':2,'d':3,
			'e':4,'f':5,'g':6,'h':7}
colsToFiles = {v: k for k,v in filesToCols.items()}

def generateOpening(engine, opening):
	#exec(f"global {opening}; opening = {opening}")
	for move_ind in range(len(opening)):
		opening[move_ind] = moveFromNotation(engine, opening[move_ind])
	return opening

def moveFromNotation(engine, notation):
	start = notation[:2]
	end = notation[2:]
	start_col = filesToCols[start[0]]
	start_row = ranksToRows[start[-1]]
	end_col = filesToCols[end[0]]
	end_row = ranksToRows[end[-1]]
	return Move((start_row, start_col), (end_row, end_col), engine.board)

def randomMove(validMoves):
	return validMoves[random.randint(0, len(validMoves)-1)]

def minMaxNoRecursionMove(engine, validMoves):
	turnMultiplier = 1 if engine.whiteToMove == True else -1
	oppMinMaxScore = CHECKMATE
	bestMove = None
	random.shuffle(validMoves)
	for playerMove in validMoves:
		engine.makeMove(playerMove)
		if engine.checkmate == True:
			oppMaxScore = -CHECKMATE
		elif engine.stalemate == True:
			oppMaxScore = STALEMATE
		else:
			oppMaxScore = -CHECKMATE
			for oppMove in engine.getValidMoves():
				engine.makeMove(oppMove)
				engine.getValidMoves()
				if engine.checkmate == True:
					score = CHECKMATE
				elif engine.stalemate == True:
					score = STALEMATE
				else:
					score = scoreMaterial(engine) * -turnMultiplier
					if score > oppMaxScore:
						oppMaxScore = score
				engine.undoMove()
		if oppMaxScore < oppMinMaxScore:
			oppMinMaxScore = oppMaxScore
			bestMove = playerMove
		engine.undoMove()
	return bestMove

def bestMove(engine, validMoves, AI):
	global nextMove, counter, pieceScore, openings, openingsName, engine_copy
	engine_copy = engine
	pieceScoreKeys = list(pieceScore.keys())
	for i in range(6):
		pieceScore[pieceScoreKeys[i]] = float(AI.iat[i, 1])
	nextMove = []
	counter = 0

	if type(whiteOpenings[0][0]) == str:
		openings = whiteOpenings + blackOpenings
		openingsName = whiteOpeningsName + blackOpeningsName
		for i in range(len(openingsName)):
			exec(f"global {openingsName[i]}, engine_copy, openingsName; {openingsName[i]} = generateOpening(engine_copy, {openingsName[i]})")

	if engine.whiteToMove == True:
		if len(engine.moveLog) == 0:
			validMoves = [m[i] for m in whiteOpenings for i in range(len(m))]
	else:
		if len(engine.moveLog) == 1:
			validMoves = [m[i] for m in blackOpenings for i in range(len(m))]

	#minMaxNoRecursionMove(engine, validMoves)
	#minMaxMove(engine, validMoves, whiteToMove, DEPTH)
	#negaMaxMove(engine, validMoves, (1 if engine.whiteToMove == True else -1), DEPTH)
	negaMaxAlphaBetaMove(engine, validMoves, -CHECKMATE, CHECKMATE, (1 if engine.whiteToMove == True else -1), AI, DEPTH)
	return nextMove

def minMaxMove(engine, validMoves, whiteToMove, depth):
	global nextMove, counter
	counter += 1
	if depth == 0:
		return scoreBoard(engine)
	random.shuffle(validMoves)
	if whiteToMove == True:
		maxScore = -CHECKMATE
		for move in validMoves:
			engine.makeMove(move)
			nextMoves = engine.getValidMoves()
			score = minMaxMove(engine, nextMoves, False, depth-1)
			if score > maxScore:
				maxScore = score
				if depth == DEPTH:
					nextMove = move
			engine.undoMove()
		return maxScore
	else:
		minScore = CHECKMATE
		for move in validMoves:
			engine.makeMove(move)
			nextMoves = engine.getValidMoves()
			score = minMaxMove(engine, nextMoves, True, depth-1)
			if score < minScore:
				minScore = score
				if depth == DEPTH:
					nextMove = move
			engine.undoMove()
		return minScore

def negaMaxMove(engine, validMoves, turnMultiplier, depth):
	global nextMove, counter
	counter += 1
	if depth == 0:
		return turnMultiplier * scoreBoard(engine)
	random.shuffle(validMoves)
	maxScore = -CHECKMATE
	for move in validMoves:
		engine.makeMove(move)
		nextMoves = engine.getValidMoves()
		score = -negaMaxMove(engine, nextMoves, -turnMultiplier, depth-1)
		if score > maxScore:
			maxScore = score
			if depth == DEPTH:
				nextMove = move
		engine.undo()
	return maxScore

def negaMaxAlphaBetaMove(engine, validMoves, alpha, beta, turnMultiplier, AI, depth):
	global nextMove, counter, pieceScore
	counter += 1
	for m in engine.getCastleMoves(engine.whiteKingLocation[0] if engine.whiteToMove == True else engine.blackKingLocation[0], engine.whiteKingLocation[1] if engine.whiteToMove == True else engine.blackKingLocation[1], [], ('w' if engine.whiteToMove else 'b')):
		validMoves.append(m)
	if depth == 0:
		bookMoveScore = 0
		for opening in chosenOpenings:
			for o in opening:
				for nM in nextMove:
					if nM == o:
						bookMoveScore = 0.9
		return turnMultiplier * (scoreBoard(engine, turnMultiplier, validMoves, AI) + (bookMoveScore if turnMultiplier == 1 else -bookMoveScore))
		#return turnMultiplier * scoreMaterial(engine)
		
	random.shuffle(validMoves)

	#Remove nonsense and really bad moves
	for m in range(len(validMoves)-1, -1, -1):
		move = validMoves[m]
		if move.pieceMoved[1] == 'Q' and len(engine.moveLog) < 10*2:
			validMoves.remove(move)
		if move.pieceMoved[1] == 'K' and move.castleMove == False:
			tmp_validMoves = validMoves.copy()
			for i in range(len(validMoves)-1, -1, -1):
				if tmp_validMoves[i].pieceMoved[1] == 'K':
					tmp_validMoves.remove(tmp_validMoves[i])
			if tmp_validMoves != []:
				validMoves.remove(move)

	maxScore = -CHECKMATE
	for move in validMoves:
		engine.makeMove(move)
		if engine.squareUnderAttack(engine.whiteKingLocation[0] if engine.whiteToMove == False else engine.blackKingLocation[0], engine.whiteKingLocation[1] if engine.whiteToMove == False else engine.blackKingLocation[1]) == False:
			nextMoves = engine.getValidMoves()
			score = -negaMaxAlphaBetaMove(engine, nextMoves, -beta, -alpha, -turnMultiplier, AI, depth-1)
			if score > maxScore:
				maxScore = score
				if depth == DEPTH:
					nextMove.append(move)
			if move.castleMove == True:
				nextMove.append(move)
		engine.undoMove()
		if maxScore > alpha:
			alpha = maxScore
		if alpha >= beta:
			break
	return maxScore
	
def scoreBoard(engine, turnMultiplier, validMoves, AI):
	engine.getValidMoves()
	if engine.checkmate == True:
		if engine.whiteToMove == True:
			return -CHECKMATE
		else:
			return CHECKMATE
	elif engine.stalemate:
		return STALEMATE

	score = 0
	for row in range(len(engine.board)):
		for col in range(len(engine.board[row])):
			square = engine.board[row][col]
			if square != '--':
				if square[0] == 'w':
					score += pieceScore[square[1]]
					score += engine.getProtections(row, col, validMoves, AI)
					score += engine.getAttacks(row, col, validMoves, AI)
					score += engine.getValidMoveForSquare(row, col, validMoves, AI)
					score += engine.castle(validMoves, AI)
					score += float(AI.iat[5,1]) if engine.squareUnderAttack(engine.blackKingLocation[0], engine.blackKingLocation[1]) == True else 0
				elif square[0] == 'b':
					score -= pieceScore[square[1]]
					score -= engine.getProtections(row, col, validMoves, AI)
					score -= engine.getAttacks(row, col, validMoves, AI)
					score -= engine.getValidMoveForSquare(row, col, validMoves, AI)
					score -= engine.castle(validMoves, AI)
					score -= float(AI.iat[5,1]) if engine.squareUnderAttack(engine.blackKingLocation[0], engine.blackKingLocation[1]) == True else 0
	score += engine.getPositionAdvantages('w', AI)
	score -= engine.getPositionAdvantages('b', AI)
	return score

def scoreMaterial(engine):
	score = 0
	for row in engine.board:
		for square in row:
			if square[0] == 'w':
				score += pieceScore[square[1]]
			elif square[0] == 'b':
				score -= pieceScore[square[1]]
	return score

def getNextMove(nextMove, engine):
	global counter, openings, chosenOpenings
	string = ""
	scores = []
	castleMoves = []
	for i in nextMove:
		if i.castleMove == True:
			castleMoves.append(i)
	if castleMoves != []:
		random.shuffle(castleMoves)
		nextMove = castleMoves
	if len(engine.moveLog) < 2:
		random.shuffle(nextMove)
	for m in nextMove[:-1]:
		string += m.getChessNotation() + (", " if m != nextMove[-2] else "")
	chosenMove = nextMove[-1]
	chosenOpenings = []
	for i in openings:
		for j in i:
			if chosenMove == j:
				chosenOpenings.append(i)
	return chosenMove, string, counter, chosenMove.getChessNotation()

In [ ]:
import pygame as pg
import ChessEngine, ChessAI
import time
import sys
import pandas as pd
import threading

sys.setrecursionlimit(10**8)

pg.init()

global GAMEOVER
GAMEOVER = False
WIDTH = HEIGHT = 512
MOVE_LOG_PANEL_WIDTH, MOVE_LOG_PANEL_HEIGHT = 250, HEIGHT
DIMENSION = 8
SQUARE_SIZE = HEIGHT / DIMENSION
MAX_FPS = 15
PIECES = {}
PIECES_NAME = ['bR','bN','bB','bQ','bK','bB','bN','bR','bP','wR','wN','wB','wQ','wK','wB','wN','wR','wP']

def loadImages():
	for piece_name in PIECES_NAME:
		PIECES[piece_name] = pg.transform.scale(pg.image.load(f"Pieces/{piece_name}.png"), (SQUARE_SIZE, SQUARE_SIZE))

def main(count):
	global GAMEOVER, playerOne, playerTwo
	playerOne = False
	playerTwo = False
	AITrain1 = AITrain2 = None
	if playerOne == False:
		AITrain1 = pd.read_excel(input("Enter AI 1 file path: "))
		if playerTwo == False:
			AITrain2 = pd.read_excel(input("Enter AI 2 file path: "))
	elif playerTwo == False:
		AITrain1 = pd.read_excel(input("Enter AI 1 file path: "))
	screen = pg.display.set_mode((WIDTH+MOVE_LOG_PANEL_WIDTH, HEIGHT))
	pg.display.set_caption('Chess')
	clock = pg.time.Clock()
	screen.fill(pg.Color("Black"))
	engine = ChessEngine.GameState()
	promotionTo = ""
	validMoves = engine.getValidMoves()
	moveMade = False
	loadImages()
	running = True
	selected = ()
	clicks = []
	for a in range(1, count+1):
		resetGame()
		running = True
		while running:
			pg.event.pump()
			playerTurn = (engine.whiteToMove == True and playerOne == True) or (engine.whiteToMove == False and playerTwo == True)
			for event in pg.event.get():
				if event.type == pg.QUIT:
					running = False
				elif event.type == pg.MOUSEBUTTONDOWN:
					if GAMEOVER == False and playerTurn:
						location = pg.mouse.get_pos()
						color = 'w' if engine.whiteToMove else 'b'
						col = int(location[0] // SQUARE_SIZE)
						row = int(location[1] // SQUARE_SIZE)
						if col >= 8:
							selected = ()
							clicks = []
						else:
							if len(clicks) == 0:
								if (engine.whiteToMove == True and 'b' in engine.board[row][col]) or (engine.whiteToMove == False and 'w' in engine.board[row][col]) or engine.board[row][col] == '--':
									selected = ()
									clicks = []
								else:
									if selected == (row, col):
										selected = ()
										clicks = []
									else:
										selected = (row, col)
										clicks.append(selected)
							else:
								if selected == (row, col):
									selected = ()
									clicks = [selected]
								else:
									selected = (row, col)
									clicks.append(selected)
									if (engine.board[row][col][0] == color and clicks[0] != selected) or (not ChessEngine.Move(clicks[0], clicks[1], engine.board) in validMoves and clicks[0] != selected):
										clicks = [clicks[-1]]
								if len(clicks) == 2 and color in engine.board[row][col]:
									clicks[0] = selected
									clicks.remove(clicks[-1])
									selected = ()
								if len(clicks) == 2:
									move = ChessEngine.Move(clicks[0], clicks[1], engine.board)
									for i in range(len(validMoves)):
										if move == validMoves[i]:
											previousScore = ChessAI.scoreBoard(engine, (-1 if engine.whiteToMove == True else 1), validMoves, AITrain1 if (playerOne and playerTwo == False) else AITrain1 if engine.whiteToMove == True else AITrain2)
											if validMoves[i].pawnPromotion == True:
												drawPromotionMenu(screen, move.endRow, move.endCol, 'w' if move.endRow == 0 else 'b')
												time.sleep(0.5)
												promoted = False
												r, c = -1, -1
												while promoted == False:
													for evt in pg.event.get():
														if evt.type == pg.MOUSEBUTTONDOWN:
															loc = pg.mouse.get_pos()
															c = int(loc[0] // SQUARE_SIZE)
															r = int(loc[1] // SQUARE_SIZE)
														if r == move.endRow and c == move.endCol:
															promotionTo = 'Q'
															promoted = True
														elif r == move.endRow-1 and move.endRow == 7 and c == move.endCol:
															promotionTo = 'R'
															promoted = True
														elif r == move.endRow-2 and move.endRow == 7 and c == move.endCol:
															promotionTo = 'B'
															promoted = True
														elif r == move.endRow-3 and move.endRow == 7 and c == move.endCol:
															promotionTo = 'N'
															promoted = True
														elif r == move.endRow+1 and move.endRow == 0 and c == move.endCol:
															promotionTo = 'R'
															promoted = True
														elif r == move.endRow+2 and move.endRow == 0 and c == move.endCol:
															promotionTo = 'B'
															promoted = True
														elif r == move.endRow+3 and move.endRow == 0 and c == move.endCol:
															promotionTo = 'N'
															promoted = True
												engine.makeMove(validMoves[i], promotionTo)
											else:
												engine.makeMove(validMoves[i])					
											score = ChessAI.scoreBoard(engine, (-1 if engine.whiteToMove == True else 1), validMoves, AITrain1 if (playerOne and playerTwo == False) else AITrain1 if engine.whiteToMove == True else AITrain2)
											print(f"{engine.moveLog[-1].getChessNotation()}: {score}")
											moveMade = True
											animate = True
											selected = ()
											clicks = []
									if moveMade == False:
										selected = ()
										clicks.remove(clicks[-1])
				elif event.type == pg.KEYDOWN:
					keys=pg.key.get_pressed()
					if keys[pg.K_LEFT]:
						engine.undoMove()
						moveMade = True
						animate = False
					if keys[pg.K_RIGHT]:
						if promotionTo != "":
							engine.redoMove(promotionTo)
						else:
							engine.redoMove()
						moveMade = True
					if keys[pg.K_SPACE]:
						resetGame()

			if GAMEOVER == False and playerTurn == False and validMoves != []:
				global AIMove
				AIMove = []
				getMove = threading.Thread(target=getAIMove, name="AIMove", args=(engine, validMoves, AITrain1, AITrain2))
				getMove.start()
				while getMove.isAlive() == True:
					pg.event.pump()
					for event in pg.event.get():
						if event.type == pg.QUIT:
							running = False
							exit()
							break
				if AIMove == []:
					AIMove = ChessAI.randomMove(validMoves)
				else:
					AIMove, string, counter, chosenMove = ChessAI.getNextMove(AIMove, engine)
				engine.makeMove(AIMove)
				score = ChessAI.scoreBoard(engine, (-1 if engine.whiteToMove == True else 1), validMoves, AITrain1 if (playerOne and playerTwo == False) else AITrain1 if engine.whiteToMove == True else AITrain2)
				print(f"\n{engine.moveLog[-1].getChessNotation()}: {score}")
				print(f"Other available moves for {chosenMove}: {string}")
				print(f"Moves generated: {counter}\n")
				moveMade = True
				animate = True
						
			if moveMade == True:
				if animate == True:
					animateMove(screen, engine.moveLog[-1], engine, clock)
				validMoves = engine.getValidMoves()
				moveMade = False
				animate = False

			updateGameState(screen, engine, validMoves, selected)
			clock.tick(MAX_FPS)
			pg.display.flip()
			if engine.checkmate == True:
				GAMEOVER = True
				engine.whiteToMove = not engine.whiteToMove
				screen.fill(pg.Color("grey"))
				screen.blit(PIECES[engine.winner], (WIDTH/2-30, HEIGHT/2-70))
				if engine.winner == "wK":
					text = 'White won!'
					#print(f"[{count}] White won!")
				else:
					text = 'Black won!'
					#print(f"[{count}] Black won!")
				pg.display.update()
				drawText(screen, text, "Black", (WIDTH / 2, HEIGHT / 2 + 50))
				time.sleep(5)
				running = False
			elif engine.stalemate == True:
				GAMEOVER = True
				engine.whiteToMove = not engine.whiteToMove
				screen.fill(pg.Color("Grey"))
				drawText(screen, "Draw - Stalemate")
				#print(f"[{count}] Draw - stalemate")
				time.sleep(5)
				running = False
			elif engine.notEnoughMaterialsToWin == True:
				GAMEOVER = True
				engine.whiteToMove = not engine.whiteToMove
				screen.fill(pg.Color("Grey"))
				drawText(screen, "Draw - Not enough materials to win")
				#print(f"[{count}] Draw - Not enough materials to win")
				time.sleep(5)
				running = False
		with open("MoveLog.txt", "a") as f:
			if engine.checkmate == True:
				if engine.winner == "wK":
					f.write(f"[{count}] White won!\n")
				elif engine.winner == "bK":
					f.write(f"[{count}] Black won!\n")
			elif engine.stalemate == True:
				f.write(f"[{count}] Draw - stalemate\n")
			for m in engine.moveLog:
				f.write(f"{m.getChessNotation()}\n")
			f.close()

def getAIMove(engine, validMoves, AITrain1, AITrain2):
	global AIMove
	AIMove = ChessAI.bestMove(engine, validMoves, AITrain1 if (playerOne and playerTwo == False) else AITrain1 if engine.whiteToMove == True else AITrain2)

def drawText(screen, text, textColor="Black", position=(WIDTH/2, HEIGHT/2), fontsize=32):
	font = pg.font.SysFont('TimesNewRoman', fontsize)
	text = font.render(text, True, pg.Color(textColor))
	textRect = text.get_rect()
	textRect.center = position
	screen.blit(text, textRect)
	pg.display.update()

def highlightSquares(screen, engine, validMoves, selected):
	if selected != ():
		r, c = selected
		if engine.board[r][c][0] == ('w' if engine.whiteToMove else 'b'):
			s = pg.Surface((SQUARE_SIZE, SQUARE_SIZE))
			s.set_alpha(131)
			s.fill(pg.Color("yellow"))
			screen.blit(s, (c*SQUARE_SIZE, r*SQUARE_SIZE))
			for move in validMoves:
				if move.startRow == r and move.startCol == c:
					if move.pieceCaptured == '--':
						s.set_alpha(100)
						s.fill(pg.Color("yellow"))
						screen.blit(s, (move.endCol*SQUARE_SIZE, move.endRow*SQUARE_SIZE))
					else:
						s.set_alpha(100)
						s.fill(pg.Color("red"))
						screen.blit(s, (move.endCol*SQUARE_SIZE, move.endRow*SQUARE_SIZE))

def resetGame():
	engine = ChessEngine.GameState()
	validMoves = engine.getValidMoves()
	selected = ()
	clicks = []
	moveMade = False
	animate = False

def highlightLastMove(screen, engine):
	if len(engine.moveLog) > 0:
		move = engine.moveLog[-1]
		s = pg.Surface((SQUARE_SIZE, SQUARE_SIZE))
		s.set_alpha(131)
		s.fill(pg.Color("yellow"))
		screen.blit(s, (move.startCol*SQUARE_SIZE, move.startRow*SQUARE_SIZE))
		screen.blit(s, (move.endCol*SQUARE_SIZE, move.endRow*SQUARE_SIZE))

def updateGameState(screen, engine, validMoves, selected):
	drawBoard(screen, engine)
	highlightLastMove(screen, engine)
	highlightSquares(screen, engine, validMoves, selected)
	drawPieces(screen, engine)
	drawMoveLog(screen, engine)

def drawMoveLog(screen, engine, fontsize=17, textColor="White"):
	font = pg.font.SysFont('TimesNewRoman', fontsize)
	padding = 5
	y = padding
	lineSpacing = 5
	movesPerRow = 2
	moveLogRect = pg.Rect(WIDTH, 0, MOVE_LOG_PANEL_WIDTH, MOVE_LOG_PANEL_HEIGHT)
	pg.draw.rect(screen, pg.Color("Black"), moveLogRect)
	log = engine.moveLog
	moveTexts = [i.getChessNotation() for i in log]
	if len(moveTexts) > 0:
		for i in range(0, len(moveTexts)):
			t = f"{(i+1)-i//2}. {moveTexts[i]}  -  {'...' if len(moveTexts) <= i+1 else moveTexts[i+1]}"
			text = font.render(t, True, pg.Color(textColor))
			if i%(movesPerRow*2) != 0 and i%2 == 0:
				textLocation = moveLogRect.move(padding + MOVE_LOG_PANEL_WIDTH / 2, y)
				screen.blit(text, textLocation)
				pg.display.update()
				y += text.get_height() + lineSpacing
			if i%(movesPerRow*2) == 0:
				textLocation = moveLogRect.move(padding, y)
				screen.blit(text, textLocation)
				pg.display.update()

def drawBoard(screen, engine):
	global playerOne, playerTwo
	colors = [pg.Color("white"), pg.Color((200, 200, 200))]
	for row in range(DIMENSION):
		for column in range(DIMENSION):
			color = colors[((row+column)%2)]
			pg.draw.rect(screen, color, pg.Rect(column*SQUARE_SIZE, row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def drawPieces(screen, engine):
	for row in range(DIMENSION):
		for column in range(DIMENSION):
			piece = engine.board[row][column]
			if piece != '--':
				screen.blit(PIECES[piece], pg.Rect(column*SQUARE_SIZE, row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def drawPromotionMenu(screen, row, col, pieceColor):
	if row == 0:
		rows = [0, 1, 2, 3]
		color = pg.Color(232, 235, 239)
	elif row == 7:
		rows = [7, 6, 5, 4]
		color = pg.Color(125, 135, 150)
	for r in rows:
		pg.draw.rect(screen, color, pg.Rect(col*SQUARE_SIZE, row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE*4))
		pg.draw.rect(screen, (0,0,0), pg.Rect(col*SQUARE_SIZE, row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE*4), 1, 5)
		screen.blit(PIECES[f'{pieceColor}Q'], pg.Rect(col*SQUARE_SIZE, rows[0]*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
		screen.blit(PIECES[f'{pieceColor}R'], pg.Rect(col*SQUARE_SIZE, rows[1]*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
		screen.blit(PIECES[f'{pieceColor}B'], pg.Rect(col*SQUARE_SIZE, rows[2]*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
		screen.blit(PIECES[f'{pieceColor}N'], pg.Rect(col*SQUARE_SIZE, rows[3]*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
	pg.display.flip()

def animateMove(screen, move, engine, clock):
	colors = [pg.Color("white"), pg.Color((200, 200, 200))]
	dR = move.endRow - move.startRow
	dC = move.endCol - move.startCol
	framePerSquare = 10
	frameCount = (abs(dR) + abs(dC)) * framePerSquare
	for frame in range(frameCount+1):
		r, c = (move.startRow + dR * frame / frameCount, move.startCol + dC * frame / frameCount)
		drawBoard(screen, engine)
		drawPieces(screen, engine)
		color = colors[(move.endRow + move.endCol) % 2]
		endSquare = pg.Rect(move.endCol*SQUARE_SIZE, move.endRow*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE)
		pg.draw.rect(screen, color, endSquare)
		if move.pieceCaptured != '--':
			if move.enpassantMove == True:
				endSquare = pg.Rect(move.endCol*SQUARE_SIZE, move.startRow*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE)
			screen.blit(PIECES[move.pieceCaptured], endSquare)
		screen.blit(PIECES[move.pieceMoved], pg.Rect(c*SQUARE_SIZE, r*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
		pg.display.flip()

if __name__ == "__main__":
	i = 1
	main(i)